## API AlphaVantage - Diario

#### Importar las librerías necesarias

In [25]:
import requests
from decouple import config
import json
import pandas as pd
import psycopg2

#### Leer la contraseña y el token desde la variable de entorno

In [26]:
my_host = config('DATABASE_HOST')
port = config('DATABASE_PORT')
password = config('DATABASE_PASSWORD')
database_name = config('DATABASE_NAME')
user = config('DATABASE_USER')
token = config('ALPHAVANTAGE_TOKEN')

#### Definir la función para obtener datos de AlphaVantage y mostrar un dataframe

In [30]:
def obtener_datos_acciones(symbols, token):
    dataframes = []
    for symbol in symbols:
        url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={token}' 
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if 'Time Series (Daily)' in data:
                time_series_data = data['Time Series (Daily)']
                df = pd.DataFrame(time_series_data).T
                df['Fecha'] = df.index
                df['symbol'] = symbol
                df = df[['Fecha', '1. open', '2. high', '3. low', '4. close', '5. volume', 'symbol']]
                df.columns = ['Fecha', 'Apertura', 'Maximo', 'Minimo', 'Cierre', 'Volumen', 'Símbolo']
                dataframes.append(df)
            else:
                print(f"No se encontraron datos para {symbol}.")
        else:
            print(f"No se pudo obtener datos para {symbol}. Código de estado: {response.status_code}")

    if dataframes:
        return pd.concat(dataframes)
    else:
        return None

# Símbolos de las acciones a consultar
symbols = ['AMZN', 'MELI', 'AAPL', 'GOOGL', 'PINS']

# Obtener datos de Alphavantage y mostrarlos en un DataFrame
df = obtener_datos_acciones(symbols, token)
if df is not None:
    print(df)
else:
    print("No se pudieron obtener datos para ninguna acción.")

No se encontraron datos para AMZN.
No se encontraron datos para MELI.
No se encontraron datos para AAPL.
No se encontraron datos para GOOGL.
No se encontraron datos para PINS.
No se pudieron obtener datos para ninguna acción.


#### Conectarse con Amazon Redshift y crear tabla (si no existe)

In [28]:

#Conectar a la base de datos
try:
    conn = psycopg2.connect(
        host=my_host, 
        port=port, 
        database=database_name, 
        user=user, 
        password=password
    )
    print("Conexión a la base de datos exitosa")

    # Crear tabla
    try:
        cur = conn.cursor()
        cur.execute("""
        CREATE TABLE IF NOT EXISTS hvzambrana_coderhouse.alphavantage (
            simbolo VARCHAR(10) DISTKEY NOT NULL,
            fecha TIMESTAMP SORTKEY NOT NULL,
            apertura FLOAT(15) NULL,
            maximo FLOAT(15) NULL,
            minimo FLOAT(15) NULL,
            cierre FLOAT(15) NULL,
            volumen FLOAT(15) NULL
        );
        """)
        conn.commit()
        print("Tabla creada exitosamente")
    except Exception as e:
            print("Error al crear la tabla")
            print(e)

except Exception as e:
    print("Error al crear la conexión a la base de datos")
    print(e)
finally:
    if conn is not None:
        conn.close()
        print("Conexión cerrada correctamente")

Conexión a la base de datos exitosa
Tabla creada exitosamente
Conexión cerrada correctamente
